In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('../data/processed/final_merged_events.csv', low_memory=False)

In [2]:
import pandas as pd

# Assuming 'date' is in a string format, convert it to datetime
data['date'] = pd.to_datetime(data['date'])

# Now you can use .dt accessor to extract date components
data['day_of_week'] = data['date'].dt.dayofweek
data['month'] = data['date'].dt.month
data['year'] = data['date'].dt.year


In [ ]:
#combined_data['rolling_7d_mean'] = combined_data.groupby(['item_id', 'store_id'])['units_sold'].rolling(window=7).mean().reset_index(level=0, drop=True)
#combined_data['rolling_30d_sum'] = combined_data.groupby(['item_id', 'store_id'])['units_sold'].rolling(window=30).sum().reset_index(level=0, drop=True)


In [ ]:
#combined_data['units_sold_lag_7'] = combined_data.groupby(['item_id', 'store_id'])['units_sold'].shift(7)


In [ ]:
import os  # Import the os module
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from category_encoders import TargetEncoder
import joblib  # Import joblib

# List of store names
store_names = ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']

# Define the groups based on your criteria
group1 = ['CA_1', 'CA_2', 'CA_3', 'CA_4']
group2 = ['TX_1', 'TX_2', 'TX_3']
group3 = ['WI_1', 'WI_2', 'WI_3']

# Initialize a dictionary to store the trained models and encoders for each group
models = {}

# Iterate over each group and filter the data accordingly
for group_idx, group in enumerate([group1, group2, group3]):
    # Filter the data for the current group
    group_data = data[data['store_id'].isin(group)]
    
    # Define features and target variable
    date_features = ['day_of_week', 'month', 'year']
    categorical_features = ['store_id', 'item_id']
    target = 'sales'
    
    # Split the data into training, validation, and test sets
    train_data, test_data = train_test_split(group_data, test_size=0.2, random_state=42)
    train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

    # Extract features and target variables for training, validation, and test sets
    X_train, y_train = train_data[date_features + categorical_features], train_data[target]
    X_val, y_val = val_data[date_features + categorical_features], val_data[target]
    X_test, y_test = test_data[date_features + categorical_features], test_data[target]

    # Initialize target encoders for categorical features
    target_encoders = {}
    for feature in categorical_features:
        target_encoder = TargetEncoder()
        X_train[feature] = target_encoder.fit_transform(X_train[feature], y_train)
        X_val[feature] = target_encoder.transform(X_val[feature])
        X_test[feature] = target_encoder.transform(X_test[feature])
        target_encoders[feature] = target_encoder

    # Initialize and train the Random Forest model on the encoded feature set
    model = RandomForestRegressor()
    model.fit(X_train, y_train)

    # Make predictions on the validation set
    val_predictions = model.predict(X_val)

    # Evaluate the model on the validation set using Mean Absolute Error (MAE)
    val_mae = mean_absolute_error(y_val, val_predictions)
    print(f'Group {group_idx + 1} - Validation MAE with Target Encoding: {val_mae}')

    # Store the trained model and encoders in the dictionary for this group
    model_and_encoders = {
        'model': model,
        'encoders': target_encoders
    }
    
    # Define the models directory
    models_dir = "../models/Randomforest"
    os.makedirs(models_dir, exist_ok=True)  # Create the directory if it doesn't exist
    
    # Define the model file path for saving
    model_file_path = os.path.join(models_dir, f'model_group_{group_idx + 1}.joblib')
    
    # Save the model to the specified file path using joblib
    joblib.dump(model_and_encoders, model_file_path)

# You can access the models and encoders for each group using the 'models' dictionary


/var/folders/dm/ly88h28j28b8nml3dg1my5bc0000gn/T/ipykernel_3643/2399735645.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[feature] = target_encoder.fit_transform(X_train[feature], y_train)
/var/folders/dm/ly88h28j28b8nml3dg1my5bc0000gn/T/ipykernel_3643/2399735645.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val[feature] = target_encoder.transform(X_val[feature])
/var/folders/dm/ly88h28j28b8nml3dg1my5bc0000gn/T/ipykernel_3643/2399735645.py:45: SettingWithCopyWarning: 
A value is tryin

Group 1 - Validation MAE with Target Encoding: 2.8351104758193864


/var/folders/dm/ly88h28j28b8nml3dg1my5bc0000gn/T/ipykernel_3643/2399735645.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[feature] = target_encoder.fit_transform(X_train[feature], y_train)
/var/folders/dm/ly88h28j28b8nml3dg1my5bc0000gn/T/ipykernel_3643/2399735645.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val[feature] = target_encoder.transform(X_val[feature])
/var/folders/dm/ly88h28j28b8nml3dg1my5bc0000gn/T/ipykernel_3643/2399735645.py:45: SettingWithCopyWarning: 
A value is tryin

In [3]:
import os
import pandas as pd
import lightgbm as lgb  # Import LightGBM
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from category_encoders import TargetEncoder
import joblib
import numpy as np

# List of store names
store_names = ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']

# Define the groups based on your criteria
group1 = ['CA_1', 'CA_2', 'CA_3', 'CA_4']
group2 = ['TX_1', 'TX_2', 'TX_3']
group3 = ['WI_1', 'WI_2', 'WI_3']

# Initialize a dictionary to store the trained models and encoders for each group
models = {}

# Initialize lists to store RMSE for each group
rmse_scores = []

# Iterate over each group and filter the data accordingly
for group_idx, group in enumerate([group1, group2, group3]):
    # Filter the data for the current group
    group_data = data[data['store_id'].isin(group)]

    # Define features and target variable
    date_features = ['day_of_week', 'month', 'year']
    categorical_features = ['store_id', 'item_id']
    target = 'sales'

    # Split the data into training, validation, and test sets
    train_data, test_data = train_test_split(group_data, test_size=0.2, random_state=42)
    train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

    # Extract features and target variables for training, validation, and test sets
    X_train, y_train = train_data[date_features + categorical_features], train_data[target]
    X_val, y_val = val_data[date_features + categorical_features], val_data[target]
    X_test, y_test = test_data[date_features + categorical_features], test_data[target]

    # Initialize target encoders for categorical features
    target_encoders = {}
    for feature in categorical_features:
        target_encoder = TargetEncoder()
        X_train[feature] = target_encoder.fit_transform(X_train[feature], y_train)
        X_val[feature] = target_encoder.transform(X_val[feature])
        X_test[feature] = target_encoder.transform(X_test[feature])
        target_encoders[feature] = target_encoder

    # Initialize and train the LightGBM model on the encoded feature set
    model = lgb.LGBMRegressor()  # Use LightGBMRegressor
    model.fit(X_train, y_train)

    # Make predictions on the validation set
    val_predictions = model.predict(X_val)

    # Evaluate the model on the validation set using Mean Absolute Error (MAE)
    val_mae = mean_absolute_error(y_val, val_predictions)
    print(f'Group {group_idx + 1} - Validation MAE with Target Encoding: {val_mae}')

    # Store the trained model and encoders in the dictionary for this group
    model_and_encoders = {
        'model': model,
        'encoders': target_encoders
    }

    # Define the models directory
    models_dir = "../models/LightGBM"  # Update the directory name
    os.makedirs(models_dir, exist_ok=True)  # Create the directory if it doesn't exist

    # Define the model file path for saving
    model_file_path = os.path.join(models_dir, f'model_group_{group_idx + 1}.joblib')

    # Save the model to the specified file path using joblib
    joblib.dump(model_and_encoders, model_file_path)

    # Make predictions on the test set
    test_predictions = model.predict(X_test)

    # Calculate RMSE on the test set
    rmse = np.sqrt(mean_squared_error(y_test, test_predictions))
    print(f'Group {group_idx + 1} - Test RMSE with Target Encoding: {rmse}')

    # Append the RMSE score to the list
    rmse_scores.append(rmse)

# Calculate the average RMSE across groups
average_rmse = np.mean(rmse_scores)
print(f'Average RMSE across groups: {average_rmse}')


/var/folders/dm/ly88h28j28b8nml3dg1my5bc0000gn/T/ipykernel_2511/945935056.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[feature] = target_encoder.fit_transform(X_train[feature], y_train)
/var/folders/dm/ly88h28j28b8nml3dg1my5bc0000gn/T/ipykernel_2511/945935056.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val[feature] = target_encoder.transform(X_val[feature])
/var/folders/dm/ly88h28j28b8nml3dg1my5bc0000gn/T/ipykernel_2511/945935056.py:49: SettingWithCopyWarning: 
A value is trying t

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.044440 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 286
[LightGBM] [Info] Number of data points in the train set: 12028182, number of used features: 5
[LightGBM] [Info] Start training from score 3.410668
Group 1 - Validation MAE with Target Encoding: 3.5032749864743495
Group 1 - Test RMSE with Target Encoding: 7.833307977149108


/var/folders/dm/ly88h28j28b8nml3dg1my5bc0000gn/T/ipykernel_2511/945935056.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[feature] = target_encoder.fit_transform(X_train[feature], y_train)
/var/folders/dm/ly88h28j28b8nml3dg1my5bc0000gn/T/ipykernel_2511/945935056.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val[feature] = target_encoder.transform(X_val[feature])
/var/folders/dm/ly88h28j28b8nml3dg1my5bc0000gn/T/ipykernel_2511/945935056.py:49: SettingWithCopyWarning: 
A value is trying t

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.032043 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 285
[LightGBM] [Info] Number of data points in the train set: 9021136, number of used features: 5
[LightGBM] [Info] Start training from score 2.929159
Group 2 - Validation MAE with Target Encoding: 3.0295616100803753
Group 2 - Test RMSE with Target Encoding: 6.948324371294832


/var/folders/dm/ly88h28j28b8nml3dg1my5bc0000gn/T/ipykernel_2511/945935056.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[feature] = target_encoder.fit_transform(X_train[feature], y_train)
/var/folders/dm/ly88h28j28b8nml3dg1my5bc0000gn/T/ipykernel_2511/945935056.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val[feature] = target_encoder.transform(X_val[feature])
/var/folders/dm/ly88h28j28b8nml3dg1my5bc0000gn/T/ipykernel_2511/945935056.py:49: SettingWithCopyWarning: 
A value is trying t

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.034933 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 285
[LightGBM] [Info] Number of data points in the train set: 9021136, number of used features: 5
[LightGBM] [Info] Start training from score 2.629241
Group 3 - Validation MAE with Target Encoding: 2.902307323203704
Group 3 - Test RMSE with Target Encoding: 6.3804925466177975
Average RMSE across groups: 7.054041631687245


In [4]:
import os
import pandas as pd
import xgboost as xgb  # Import XGBoost
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from category_encoders import TargetEncoder
import joblib
import numpy as np

# List of store names
store_names = ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']

# Define the groups based on your criteria
group1 = ['CA_1', 'CA_2', 'CA_3', 'CA_4']
group2 = ['TX_1', 'TX_2', 'TX_3']
group3 = ['WI_1', 'WI_2', 'WI_3']

# Initialize a dictionary to store the trained models and encoders for each group
models = {}

# Initialize lists to store RMSE for each group
rmse_scores = []

# Iterate over each group and filter the data accordingly
for group_idx, group in enumerate([group1, group2, group3]):
    # Filter the data for the current group
    group_data = data[data['store_id'].isin(group)]

    # Define features and target variable
    date_features = ['day_of_week', 'month', 'year']
    categorical_features = ['store_id', 'item_id']
    target = 'sales'

    # Split the data into training, validation, and test sets
    train_data, test_data = train_test_split(group_data, test_size=0.2, random_state=42)
    train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

    # Extract features and target variables for training, validation, and test sets
    X_train, y_train = train_data[date_features + categorical_features], train_data[target]
    X_val, y_val = val_data[date_features + categorical_features], val_data[target]
    X_test, y_test = test_data[date_features + categorical_features], test_data[target]

    # Initialize target encoders for categorical features
    target_encoders = {}
    for feature in categorical_features:
        target_encoder = TargetEncoder()
        X_train[feature] = target_encoder.fit_transform(X_train[feature], y_train)
        X_val[feature] = target_encoder.transform(X_val[feature])
        X_test[feature] = target_encoder.transform(X_test[feature])
        target_encoders[feature] = target_encoder

    # Initialize and train the XGBoost model on the encoded feature set
    model = xgb.XGBRegressor()  # Use XGBRegressor
    model.fit(X_train, y_train)

    # Make predictions on the validation set
    val_predictions = model.predict(X_val)

    # Evaluate the model on the validation set using Mean Absolute Error (MAE)
    val_mae = mean_absolute_error(y_val, val_predictions)
    print(f'Group {group_idx + 1} - Validation MAE with Target Encoding: {val_mae}')

    # Store the trained model and encoders in the dictionary for this group
    model_and_encoders = {
        'model': model,
        'encoders': target_encoders
    }

    # Define the models directory
    models_dir = "../models/XGBoost"  # Update the directory name
    os.makedirs(models_dir, exist_ok=True)  # Create the directory if it doesn't exist

    # Define the model file path for saving
    model_file_path = os.path.join(models_dir, f'model_group_{group_idx + 1}.joblib')

    # Save the model to the specified file path using joblib
    joblib.dump(model_and_encoders, model_file_path)

    # Make predictions on the test set
    test_predictions = model.predict(X_test)

    # Calculate RMSE on the test set
    rmse = np.sqrt(mean_squared_error(y_test, test_predictions))
    print(f'Group {group_idx + 1} - Test RMSE with Target Encoding: {rmse}')

    # Append the RMSE score to the list
    rmse_scores.append(rmse)

# Calculate the average RMSE across groups
average_rmse = np.mean(rmse_scores)
print(f'Average RMSE across groups: {average_rmse}')


/var/folders/dm/ly88h28j28b8nml3dg1my5bc0000gn/T/ipykernel_2511/2942416930.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[feature] = target_encoder.fit_transform(X_train[feature], y_train)
/var/folders/dm/ly88h28j28b8nml3dg1my5bc0000gn/T/ipykernel_2511/2942416930.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val[feature] = target_encoder.transform(X_val[feature])
/var/folders/dm/ly88h28j28b8nml3dg1my5bc0000gn/T/ipykernel_2511/2942416930.py:49: SettingWithCopyWarning: 
A value is tryin

Group 1 - Validation MAE with Target Encoding: 3.4851860637365495
Group 1 - Test RMSE with Target Encoding: 7.8817536483258435


/var/folders/dm/ly88h28j28b8nml3dg1my5bc0000gn/T/ipykernel_2511/2942416930.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[feature] = target_encoder.fit_transform(X_train[feature], y_train)
/var/folders/dm/ly88h28j28b8nml3dg1my5bc0000gn/T/ipykernel_2511/2942416930.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val[feature] = target_encoder.transform(X_val[feature])
/var/folders/dm/ly88h28j28b8nml3dg1my5bc0000gn/T/ipykernel_2511/2942416930.py:49: SettingWithCopyWarning: 
A value is tryin

Group 2 - Validation MAE with Target Encoding: 3.0255307006170726
Group 2 - Test RMSE with Target Encoding: 7.01639161125772


/var/folders/dm/ly88h28j28b8nml3dg1my5bc0000gn/T/ipykernel_2511/2942416930.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[feature] = target_encoder.fit_transform(X_train[feature], y_train)
/var/folders/dm/ly88h28j28b8nml3dg1my5bc0000gn/T/ipykernel_2511/2942416930.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val[feature] = target_encoder.transform(X_val[feature])
/var/folders/dm/ly88h28j28b8nml3dg1my5bc0000gn/T/ipykernel_2511/2942416930.py:49: SettingWithCopyWarning: 
A value is tryin

Group 3 - Validation MAE with Target Encoding: 2.878253019521291
Group 3 - Test RMSE with Target Encoding: 6.375807490342143
Average RMSE across groups: 7.091317583308569


In [1]:
import pandas as pd
import numpy as np
test_data = pd.read_csv('../data/processed/test_final_merged.csv', low_memory=False)


In [2]:
test_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,date,units_sold,wm_yr_wk,sell_price,sales,event_name,event_count
0,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2015-04-19,1,11512,2.24,2.24,NoEvent,0.0
1,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2015-04-20,0,11512,2.24,0.00,NoEvent,0.0
2,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2015-04-21,0,11512,2.24,0.00,NoEvent,0.0
3,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2015-04-22,0,11512,2.24,0.00,NoEvent,0.0
4,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2015-04-23,1,11512,2.24,2.24,NoEvent,0.0


In [5]:
import pandas as pd

# Assuming 'date' is in a string format, convert it to datetime
test_data['date'] = pd.to_datetime(test_data['date'])

# Now you can use .dt accessor to extract date components
test_data['day_of_week'] = test_data['date'].dt.dayofweek
test_data['month'] = test_data['date'].dt.month
test_data['year'] = test_data['date'].dt.year


In [6]:
test_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,date,units_sold,wm_yr_wk,sell_price,sales,event_name,event_count,day_of_week,month,year
0,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2015-04-19,1,11512,2.24,2.24,NoEvent,0.0,6,4,2015
1,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2015-04-20,0,11512,2.24,0.00,NoEvent,0.0,0,4,2015
2,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2015-04-21,0,11512,2.24,0.00,NoEvent,0.0,1,4,2015
3,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2015-04-22,0,11512,2.24,0.00,NoEvent,0.0,2,4,2015
4,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2015-04-23,1,11512,2.24,2.24,NoEvent,0.0,3,4,2015


In [10]:
import pandas as pd
import joblib
from sklearn.metrics import mean_squared_error
import numpy as np
import random

# Define the store names and groups
store_names = ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']
group1 = ['CA_1', 'CA_2', 'CA_3', 'CA_4']
group2 = ['TX_1', 'TX_2', 'TX_3']
group3 = ['WI_1', 'WI_2', 'WI_3']

# Filter the data for the year 2016
test_data_2016 = test_data[test_data['year'] == 2016]

# Define the number of random days to select
num_random_days = 30

# Get a random sample of day indices from the 2016 data
random_day_indices = random.sample(test_data_2016.index.tolist(), num_random_days)

# Iterate through each randomly selected day
for index in random_day_indices:
    row = test_data_2016.loc[index]
    
    # Determine the model_group based on the store_id
    store_id = row['store_id']
    if store_id in group1:
        model_group = 1
    elif store_id in group2:
        model_group = 2
    elif store_id in group3:
        model_group = 3
    else:
        # Handle the case when the store_id doesn't match any group
        model_group = None

    if model_group is not None:
        # Load the trained model for the determined model_group
        model_file_path = f"../models/LightGBM/model_group_{model_group}.joblib"
        loaded_model = joblib.load(model_file_path)

        # Prepare the input data for batch prediction
        input_data = pd.DataFrame({
            'day_of_week': [row['day_of_week']],
            'month': [row['month']],
            'year': [row['year']],
            'store_id': [row['store_id']],
            'item_id': [row['item_id']]
        })

        # Perform target encoding on categorical features using the loaded encoders
        for feature, encoder in loaded_model['encoders'].items():
            input_data[feature] = encoder.transform(input_data[feature])

        # Make batch predictions using the loaded model
        predicted_sales_batch = loaded_model['model'].predict(input_data)

        # Calculate the squared difference for the batch
        squared_diff_batch = (row['sales'] - predicted_sales_batch[0]) ** 2

        # Calculate the RMSE for the selected random day
        rmse_batch = np.sqrt(squared_diff_batch)

        # Print the RMSE for the selected random day
        print(f"LightGBM RMSE for {row['date']} in {row['store_id']}: {rmse_batch}")


LightGBM RMSE for 2016-01-11 00:00:00 in WI_3: 0.48677423170428646
LightGBM RMSE for 2016-03-24 00:00:00 in TX_1: 3.1138865529354
LightGBM RMSE for 2016-05-10 00:00:00 in CA_3: 5.3578788478683395
LightGBM RMSE for 2016-04-12 00:00:00 in TX_2: 0.9024457514828662
LightGBM RMSE for 2016-04-24 00:00:00 in CA_3: 30.815673334087982
LightGBM RMSE for 2016-01-30 00:00:00 in TX_2: 0.7795138958089556
LightGBM RMSE for 2016-04-28 00:00:00 in WI_3: 3.5203834390956046
LightGBM RMSE for 2016-03-27 00:00:00 in WI_2: 1.694393400391195
LightGBM RMSE for 2016-04-04 00:00:00 in CA_3: 5.8353785330305135
LightGBM RMSE for 2016-03-04 00:00:00 in WI_1: 2.474678736183305
LightGBM RMSE for 2016-01-19 00:00:00 in WI_1: 3.8064670854149467
LightGBM RMSE for 2016-04-07 00:00:00 in CA_2: 9.098840376913074
LightGBM RMSE for 2016-02-21 00:00:00 in CA_4: 0.9497030738231784
LightGBM RMSE for 2016-02-07 00:00:00 in CA_4: 1.9663429598712536
LightGBM RMSE for 2016-01-13 00:00:00 in CA_1: 3.620018836961174
LightGBM RMSE fo

In [13]:
import pandas as pd
import joblib
from sklearn.metrics import mean_squared_error
import numpy as np
import random

# Define the store names and groups
store_names = ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']
group1 = ['CA_1', 'CA_2', 'CA_3', 'CA_4']
group2 = ['TX_1', 'TX_2', 'TX_3']
group3 = ['WI_1', 'WI_2', 'WI_3']

# Filter the data for the year 2016
test_data_2016 = test_data[test_data['year'] == 2016]

# Define the number of random days to select
num_random_days = 30

# Get a random sample of day indices from the 2016 data
random_day_indices = random.sample(test_data_2016.index.tolist(), num_random_days)

# Iterate through each randomly selected day
for index in random_day_indices:
    row = test_data_2016.loc[index]
    
    # Determine the model_group based on the store_id
    store_id = row['store_id']
    if store_id in group1:
        model_group = 1
    elif store_id in group2:
        model_group = 2
    elif store_id in group3:
        model_group = 3
    else:
        # Handle the case when the store_id doesn't match any group
        model_group = None

    if model_group is not None:
        # Load the trained model for the determined model_group
        model_file_path = f"../models/XGBoost/model_group_{model_group}.joblib"
        loaded_model = joblib.load(model_file_path)

        # Prepare the input data for batch prediction
        input_data = pd.DataFrame({
            'day_of_week': [row['day_of_week']],
            'month': [row['month']],
            'year': [row['year']],
            'store_id': [row['store_id']],
            'item_id': [row['item_id']]
        })

        # Perform target encoding on categorical features using the loaded encoders
        for feature, encoder in loaded_model['encoders'].items():
            input_data[feature] = encoder.transform(input_data[feature])

        # Make batch predictions using the loaded model
        predicted_sales_batch = loaded_model['model'].predict(input_data)

        # Calculate the squared difference for the batch
        squared_diff_batch = (row['sales'] - predicted_sales_batch[0]) ** 2

        # Calculate the RMSE for the selected random day
        rmse_batch = np.sqrt(squared_diff_batch)

        # Print the RMSE for the selected random day
        print(f"XGBoost RMSE for {row['date']} in {row['store_id']}: {rmse_batch}")


XGBoost RMSE for 2016-03-22 00:00:00 in CA_1: 1.4539107084274292
XGBoost RMSE for 2016-03-29 00:00:00 in CA_2: 1.3406847715377808
XGBoost RMSE for 2016-04-29 00:00:00 in TX_2: 2.008944511413574
XGBoost RMSE for 2016-03-05 00:00:00 in CA_4: 1.8767356872558594
XGBoost RMSE for 2016-03-12 00:00:00 in CA_2: 0.7277894616127014
XGBoost RMSE for 2016-01-22 00:00:00 in CA_1: 0.807964563369751
XGBoost RMSE for 2016-03-24 00:00:00 in WI_3: 0.6300606727600098
XGBoost RMSE for 2016-02-09 00:00:00 in TX_2: 0.11622856140136717
XGBoost RMSE for 2016-04-23 00:00:00 in CA_3: 10.2645845413208
XGBoost RMSE for 2016-02-26 00:00:00 in WI_2: 7.821893692016602
XGBoost RMSE for 2016-01-20 00:00:00 in TX_1: 0.36802172660827637
XGBoost RMSE for 2016-03-18 00:00:00 in WI_2: 1.4428066635131835
XGBoost RMSE for 2016-04-05 00:00:00 in WI_2: 1.9420690536499023
XGBoost RMSE for 2016-01-11 00:00:00 in WI_1: 2.38731728553772
XGBoost RMSE for 2016-04-10 00:00:00 in TX_3: 1.0643382644653316
XGBoost RMSE for 2016-05-22 00